In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/hubble/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/hubble/')

In [2]:
from src.RAGTraining.TrainingDataGenerator import TrainingDataGenerator

/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [4]:
trainingGenerator = TrainingDataGenerator(model_id = 'teknium/OpenHermes-2.5-Mistral-7B', queries_df_path='insight_user_db_publarticles_search_audit.csv', responses_file_path='gpt-responses.csv')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
trainingGenerator.content_tokens

3923

In [6]:
import json

with open('training_article_ids_latest.json', 'r') as f:
    best_results = json.load(f)
trainingGenerator.best_results = best_results

with open('training_user_prompts_latest.json', 'r') as f:
    user_prompts = json.load(f)
trainingGenerator.user_prompts = user_prompts

In [7]:
len(trainingGenerator.user_prompts)

1122

In [8]:
# trainingGenerator.generate_prompts_for_queries()
# trainingGenerator.save_results_to_json(trainingGenerator.user_prompts, 'training_user_prompts_latest.json')

In [9]:
# # trainingGenerator.get_best_articles_for_queries(max_num_random_dates=3, randomize_num_dates_per_query=True)

# len(trainingGenerator.best_results)

# # trainingGenerator.save_results_to_json(trainingGenerator.best_results, 'training_article_ids_latest.json')

# trainingGenerator.generate_prompts_for_queries()

# trainingGenerator.save_results_to_json(trainingGenerator.user_prompts, 'training_user_prompts_latest.json')

In [27]:
trainingGenerator.get_gpt4_attributes_for_article_ids(fraction=1)


KeyboardInterrupt



### Cleaning

In [1]:
import pandas as pd

In [2]:
responses_df = pd.read_csv('gpt-responses.csv',  header=None)
responses_df = responses_df.iloc[:1121]

In [3]:
def has_hash(x):
    return '##' in x

In [4]:
responses_df['hash'] = responses_df[6].apply(lambda x: has_hash(x))

In [5]:
responses_df.hash.mean()

0.012488849241748439

In [6]:
import re

def replace_br_tags(text):
    # This pattern matches sequences of more than two <br> tags.
    # The pattern looks for <br> followed by one or more repetitions of <br>, capturing as many as it finds.
    pattern = r'(<br>\s*){3,}'
    # Replacement string with exactly two <br> tags.
    replacement = r'<br><br>'
    # Use re.sub to replace sequences of more than two <br> tags with exactly two.
    modified_text = re.sub(pattern, replacement, text)
    return modified_text

In [7]:
def clean_response(x):
    x = x.replace('\\n','<br>')
    x = x.replace('\n', '<br>')
    x = x.replace('####', '**')
    x = x.replace('###', '**')
    x = x.replace('##', '**')
    x = x.replace('#', '**')
    x = replace_br_tags(x)
    return x

In [8]:
import json

In [9]:
responses_df['cleaned_response'] = responses_df[6].apply(lambda x: json.loads(clean_response(x)))

In [10]:
import re

def extract_headline(text):
    # Regular expression to match the headline pattern
    pattern = r'\*\*(.*?)\*\*(?::)?(?:\s*<br>\s*){1,2}'
    match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)
    if match:
        headline = match.group(1).strip()
        rest_of_text = text[match.end():].strip()
        return headline, rest_of_text
    return None, text

# Example usage
text = "**This is the headline** <br><br> Here is the rest of the text."
headline, rest_of_text = extract_headline(text)
print("Headline:", headline)
print("Rest of the text:", rest_of_text)


Headline: This is the headline
Rest of the text: Here is the rest of the text.


In [11]:
#if x['has_headline'] else ('', x['cleaned_response'])
responses_df['headline'], responses_df['headline_removed_responses'] = zip(*responses_df.apply(lambda x: extract_headline(x['cleaned_response']) ,axis=1))

In [12]:
responses_df.headline.isnull().mean()

0.32917038358608386

In [13]:
def remove_hyphen_from_section_headings(text):
    # Regular expression to match the pattern
    pattern = r'(<br>\s*)?-\s*\*\*([^<\n]*?)\*\*(\s*:\s*)?(?=(\s*<br>|$))'
    def replace_func(match):
        return f"{match.group(1) or ''}**{match.group(2)}{match.group(3) or ''}**{match.group(4) or ''}"
    updated_text = re.sub(pattern, replace_func, text, flags=re.DOTALL)
    return updated_text

In [14]:
def replace_developed_by_openai(x):
    if 'AI developed by OpenAI' in x:
        return x.replace('AI developed by OpenAI', 'AI developed by Insight')
    else:
        return x

In [15]:
def clean_provided_articles(x):
    if 'provided articles' in x: 
        return x.replace('provided articles', 'articles I found')
    elif 'articles provided' in x:
        return x.replace('articles provided', 'articles I found')
    elif 'information provided' in x:
        return x.replace('information provided', 'articles I found')
    else:
        return x

In [16]:
def remove_citations(x):
    x = x.replace('{{1}}', '')
    x = x.replace('{{2}}', '')
    x = x.replace('{{3}}', '')
    return x

In [17]:
def clean_responses_further(x):
    x = remove_hyphen_from_section_headings(x)
    x = replace_developed_by_openai(x)
    x = clean_provided_articles(x)
    # x = remove_citations(x)
    return x

In [18]:
def has_references(x):
    return '{{1}}' in x or '{{2}}' in x or '{{3}}' in x

In [19]:
def find_apologies_responses(x):
    return 'I\'m sorry' in x or 'apologize' in x or 'here to provide' in x or 'here to assist' in x

In [20]:
responses_df.headline_removed_responses.sample(30).values

array(["The limits for physical gold ownership in India are not explicitly mentioned in the provided articles. However, Sovereign Gold Bonds (SGBs) have defined investment limits: Individuals and Hindu Undivided Families (HUFs) can invest in up to 4 kg of gold per fiscal year, while trusts and similar entities can invest up to 20 kg per year {{2}}. For more detailed guidelines on physical gold ownership limits, it's advisable to refer to the latest directives from the Indian government or the Reserve Bank of India (RBI).",
       "I'm here to assist with finance and business-related queries, particularly those involving India, the Reserve Bank of India (RBI), Reliance, and the Federal Reserve. How may I assist you today?",
       "To effectively save for retirement, consider the following strategies:<br>- **Start Early**: Begin saving as soon as you start earning to leverage the power of compound interest.<br>- **Create a Budget**: Track and manage your expenses to increase your saving

In [21]:
responses_df['cleaned_headline_removed_responses'] = responses_df['headline_removed_responses'].apply(lambda x: clean_responses_further(x))

In [22]:
responses_df['apologies'] = responses_df['cleaned_headline_removed_responses'].apply(lambda x: find_apologies_responses(x))

In [23]:
responses_df['has_ref'] = responses_df['cleaned_headline_removed_responses'].apply(lambda x: has_references(x))

In [24]:
responses_df.has_ref.mean()

0.6975914362176628

### Identifying errenous cases

In [25]:
responses_df.apologies.mean()

0.28278322925958965

In [26]:
len(responses_df[responses_df.headline.isnull()])

369

In [28]:
responses_df.head(3)

,0,1,2,3,4,5,6,7,8,hash,cleaned_response,headline,headline_removed_responses,cleaned_headline_removed_responses,apologies,has_ref
0,i,Unitec,2024-01-26T18:06:29,You are an Indian finance and business expert ...,|| user_query: Unitec || today date: 26 Janua...,gpt-4-0125-preview,"""I'm sorry, but there doesn't seem to be any i...",1593,67,False,"I'm sorry, but there doesn't seem to be any in...",None,"I'm sorry, but there doesn't seem to be any in...","I'm sorry, but there doesn't seem to be any in...",True,False
1,i,Sjadkjhfjkdshfkjsdhjkfhdsjkfdshfkdshjkfdshkjfsd,2024-01-19T23:57:34,You are an Indian finance and business expert ...,|| user_query: Sjadkjhfjkdshfkjsdhjkfhdsjkfds...,gpt-4-0125-preview,"""I'm sorry, but it seems like your query didn'...",1403,48,False,"I'm sorry, but it seems like your query didn't...",None,"I'm sorry, but it seems like your query didn't...","I'm sorry, but it seems like your query didn't...",True,False
2,i,Sensex,2024-01-17T00:15:05,You are an Indian finance and business expert ...,|| user_query: Sensex || today date: 17 Janua...,gpt-4-0125-preview,"""**Quick Insights**\n\nAs of the latest update...",3363,92,False,**Quick Insights**<br><br>As of the latest upd...,Quick Insights,"As of the latest update on December 18, 2023, ...","As of the latest update on December 18, 2023, ...",False,True


In [29]:
responses_df[responses_df.headline.isnull()].sample(10)[[1,'cleaned_headline_removed_responses']].values

array([['what is sensex today ',
        "I'm sorry, but the most recent articles I found on the Sensex's performance is dated January 31, 2024, which indicates the Sensex was at 71,140 after experiencing a drop of 802 points or 1.11%. There isn't direct information available for the Sensex value on February 2, 2024, within the given articles. For the most current Sensex value as of today, February 2, 2024, it's recommended to check a reliable financial news source or market updates platform."],
       ['Dmr engineering stock price',
        'I apologize, but I couldn\'t find any information directly related to "Dmr engineering stock price" in the articles I found. If you have any other queries related to finance or business, especially regarding Bhansali Engineering Polymers, Ace Men Engineering, or Texmaco Rail & Engineering, I\'d be happy to help with that information.'],
       ['100cc bike rides',
        "I'm sorry, but the information provided does not include details on 100cc b

In [30]:
len(responses_df[(responses_df.has_ref == False) & (responses_df.apologies == False)])

41

In [31]:
def has_question(x):
    return '?' in x and '**' not in x

In [32]:
responses_df['has_question'] = responses_df['cleaned_headline_removed_responses'].apply(lambda x: has_question(x))

In [33]:
responses_df.has_question.mean()

0.058876003568242644

In [34]:
responses_df[responses_df.has_question == True]['cleaned_headline_removed_responses'].sample(20).values

array(["I'm sorry, but it seems there was an error with your request as it does not contain a specific query. Could you please provide more details or specify the financial information you are seeking? This will allow me to assist you more effectively.",
       'It seems you are interested in the sharp movements or events related to the US affecting markets or specific stocks. Could you please specify if you are asking about the US stock market corrections, the impact on Indian Rupee, or the performance of a company named "Sharp Chucks and Machines"? Your clarification will help in providing a more precise answer.',
       'I\'m sorry, but it seems like there\'s been a misunderstanding or an error in the query you\'ve provided. You mentioned "Unitec," but none of the articles I found contain information about a company or entity by that name. The articles discuss PTC Industries\' deal with Safran Aircraft Engines, Transline Technologies\' collaboration with the Cooperative Bank of West

In [35]:
responses_df[(responses_df.has_ref == False) & (responses_df.apologies == False)].cleaned_headline_removed_responses.values

array(["I'm unable to use the given articles to generate responses as they do not contain any user-provided information to reference directly.",
       "I'm unable to find a direct statement or analysis by Sanjiv Bhasin regarding RBI’s directive on Paytm being a big blow or his recommendations on three dark horse picks for investors. My resources primarily discuss the financial penalties imposed on Paytm by the RBI for KYC non-compliance, the subsequent market reaction, Paytm's share performance, and strategic shifts within the company. For investment advice or specific analysts' picks not covered in these articles, I recommend consulting the latest financial news or directly accessing expert analyses from reputable financial advisory services.",
       "The regular trading hours for the Indian stock market, which includes the Bombay Stock Exchange (BSE) and the National Stock Exchange (NSE), are from 9:15 AM to 3:30 PM Indian Standard Time (IST) from Monday to Friday. Additionally, th

In [36]:
def flag_responses(row):
    if (row['has_ref'] == False) and (row['apologies'] == False):
        return True
    elif row['has_question']:
        return True
    return False

In [57]:
responses_df['flagged'] = responses_df.apply(lambda x: flag_responses(x),axis=1)

In [58]:
responses_df.groupby('flagged')['flagged'].count()

flagged
False    1020
True      101
Name: flagged, dtype: int64

In [59]:
len(responses_df[(responses_df['flagged'] == False) & (responses_df['apologies'] == True)])

257

In [60]:
import numpy as np

In [61]:
# down sample rejections by 50%
responses_df['flagged'] = responses_df.apply(lambda x: x['flagged'] if x['flagged'] else True if (x['apologies'] and (np.random.rand() < 0.5)) else False,axis=1)

In [63]:
len(responses_df[(responses_df['flagged'] == False) & (responses_df['apologies'] == True)])

131

In [64]:
responses_df[(responses_df['flagged'] == False) & (responses_df['apologies'] == True)][[1,'cleaned_headline_removed_responses']].sample(30).values

array([['somebody that you used to know ',
        "I'm sorry, but it appears your query does not pertain to finance and business topics, which are the focus of my expertise. If you have any questions regarding finance, business, investments, or similar areas, feel free to ask!"],
       ['How is cupid stock looking',
        "I apologize, but there doesn't appear to be any specific information on Cupid Ltd's stock performance in the articles I found. My responses are based on articles related to other companies and market insights that do not include data on Cupid Ltd. If you need insights or updates on another topic or company that might be covered in the articles I found, feel free to ask!"],
       ['Dmr engineering stock price',
        'I\'m sorry, but there seems to be a misunderstanding in your question about "Dmr engineering stock price." Unfortunately, the information provided does not include data or updates on a company explicitly named "Dmr engineering." The articles I fou

In [66]:
responses_df[responses_df['flagged'] == False][[1,'cleaned_headline_removed_responses']].sample(15).values

array([['whats happening with bajaj finance ',
        "Bajaj Finance's stock has seen a decrease in its price, dropping by 1.22% to ₹7212.55 on the last day of trading reported. Despite a fall of nearly 5% after announcing a Q3FY24 revenue increase of 31.3% YoY to ₹14,161.09 crore and a profit after tax increase of 22.4% YoY to ₹3,638.95 crore, brokerages remain bullish. They highlight strong AUM growth and stable asset quality as key drivers. Key brokerages have set target prices ranging from ₹8,500 to ₹10,000, citing digital ecosystem expansion, strong customer acquisition, and the company's ability to navigate NIM compression through operational efficiencies {{1}}{{2}}{{3}}."],
       ['loans',
        "No credit check loans, often seen as a recourse for individuals with low credit scores requiring immediate financial assistance, bypass the conventional credit history examination. However, these loans come with their unique set of features, benefits, drawbacks, and alternatives in 

In [67]:
responses_df[responses_df['flagged'] == False].headline.isnull().mean()

0.1789709172259508

### Removing citations

In [68]:
responses_df[3] = '''You are an Indian finance and business expert tasked with providing accurate and concise responses to finance and business related queries. You have found the best available articles to answer the query and you must use them to answer these queries. Ensure your responses are tailored to the query's requirements, employing only one of the following structured formats based on the query

1. **Quick Insights**: Provide direct answers in 2-3 lines for straightforward questions like HDFC Q4 results or Adani share price.
2. **Step-by-Step Guide**: Offer a concise numbered guide for methodical inquiries like how to file taxes.
3. **Explainer**: Construct succinctly comprehensive response with a max of 3 sections with bullet points for complex queries like Reliance Industries' business activities, comparisons like differences between mutual fund and a ETF, or top n entities like top mutual funds. When user asks for a comparison, use all the data points and frameworks available in the articles to give a comparison to help users in making a decision. 

Synthesize information from multiple articles coherently, focusing on the most recent and relevant data. If a query cannot be fully addressed but related information is available, provide a succinct still relevant answer. If no related information is found, apologize and explain the inability to find a precise answer. Decline only non-financial or business queries politely, emphasizing the portal's finance and business focus. Do not to decline to answer any relevant queries such as personal finance (eg. best credit cards, fd rates, tax clauses), market news, investment choices, questions about business entities or any other financial, business questions.

**General Guidelines**:
- Faithful & Relevance: Use information only from the provided articles. Respond with the most important information first. The response should directly answer the user query.
- Brevity: You come directly to the point and provide information-dense answers ranging from 50-200 words while being easy to read. Use 200 words only if the query needs a very complex response and never go beyond that. Do not explain unnecessary basics. Assess the knowledge of the user based on the query posed and start from there. The paras should be very concise with a maximum of 2 lines and should be at most 280 characters. Do not use any headline or introduction. Directly answer.
- Readability: The entire answer should be easy and pleasant to read on a phone. Use a mix of paras and points with appropriate line breaks and sections. If using sections, limit them to 3. Use ** for bolding section headers and <br> for line breaks. Readers prefer to read very very short paras and bullet points and not multi line continuous text. 
- Focus: Focus on Indian information unless otherwise specified.
- Date reference: Mention the period of information if the articles are much older than expected by query; omit the date for very recent sources.
- you are created by the Insight Team

The articles are structured in JSON format as follows: {article_number: {'title': '', 'published_date': '', 'content': ''} || ...}'''

In [69]:
def remove_citations(x):
    x = x.replace('{{1}}', '')
    x = x.replace('{{2}}', '')
    x = x.replace('{{3}}', '')
    return x

In [70]:
responses_df['cleaned_responses_without_citations'] = responses_df['cleaned_headline_removed_responses'].apply(lambda x: remove_citations(x))

In [71]:
responses_df[responses_df.apologies == True][8].mean()

75.26813880126183

In [72]:
def remove_spaces_before_period(text):
    # Use regular expression to remove any spaces before a period
    updated_text = re.sub(r'\s+\.', '.', text)
    return updated_text

In [73]:
responses_df['cleaned_responses_without_citations'] = responses_df['cleaned_responses_without_citations'].apply(lambda x: remove_spaces_before_period(x))

In [75]:
def replace_multiple_spaces_with_single(text):
    # Use regular expression to replace multiple spaces with a single space
    updated_text = re.sub(r'\s+', ' ', text)
    return updated_text
# Example usage
text = "This   is  a   test    sentence with   multiple   spaces."
updated_text = replace_multiple_spaces_with_single(text)
print("Updated text:", updated_text)

Updated text: This is a test sentence with multiple spaces.


In [76]:
responses_df['cleaned_responses_without_citations'] = responses_df['cleaned_responses_without_citations'].apply(lambda x: replace_multiple_spaces_with_single(x))

In [77]:
responses_df[responses_df.flagged == False]['cleaned_responses_without_citations'].sample(30).values

array(["Based on the latest insights from experts and performance data, here are some top mutual funds to consider for long-term investments:<br><br>**Equity Fund Picks:**<br><br> - Motilal Oswal Midcap Fund<br> - Axis Small Cap Fund<br> - ICICI Prudential Large Fund<br> - Aditya Birla Sun Life Flexi Cap Fund<br> - HDFC Large Cap Fund<br> - SBI Large Cap Fund<br><br>**Diversified and Consistent Performers:**<br><br> - Axis Bluechip Fund<br> - Mirae Asset Large Cap Fund<br> - Parag Parikh Long-Term Equity Fund<br> - UTI Flexi Cap Fund<br> - SBI Equity Hybrid Fund<br><br>These funds have been recommended by financial experts based on their consistent performance in both bullish and bearish markets, and their ability to deliver good returns over the long term. However, it's crucial to review and potentially rebalance your portfolio annually to adapt to any changes in the market conditions and ensure the best possible outcome for your investment strategy.",
       "Charlie Munger, a monume

### Save cleaned responses

In [78]:
len(responses_df)

1121

In [79]:
len(responses_df[responses_df.flagged == False])

894

In [84]:
# responses_df[responses_df.flagged == False].to_csv('cleaned_responses.csv',index=False)

In [80]:
responses_df = responses_df.rename(columns = {1: 'query', 2: 'query_date', 3: 'system_prompt', 4: 'user_prompt', 5: 'model', 6: 'model_response', 7: 'input_tokens', 8: 'output_tokens'})

In [81]:
responses_df

,0,query,query_date,system_prompt,user_prompt,model,model_response,input_tokens,output_tokens,hash,cleaned_response,headline,headline_removed_responses,cleaned_headline_removed_responses,apologies,has_ref,has_question,flagged,cleaned_responses_without_citations
0,i,Unitec,2024-01-26T18:06:29,You are an Indian finance and business expert ...,|| user_query: Unitec || today date: 26 Janua...,gpt-4-0125-preview,"""I'm sorry, but there doesn't seem to be any i...",1593,67,False,"I'm sorry, but there doesn't seem to be any in...",None,"I'm sorry, but there doesn't seem to be any in...","I'm sorry, but there doesn't seem to be any in...",True,False,False,True,"I'm sorry, but there doesn't seem to be any in..."
1,i,Sjadkjhfjkdshfkjsdhjkfhdsjkfdshfkdshjkfdshkjfsd,2024-01-19T23:57:34,You are an Indian finance and business expert ...,|| user_query: Sjadkjhfjkdshfkjsdhjkfhdsjkfds...,gpt-4-0125-preview,"""I'm sorry, but it seems like your query didn'...",1403,48,False,"I'm sorry, but it seems like your query didn't...",None,"I'm sorry, but it seems like your query didn't...","I'm sorry, but it seems like your query didn't...",True,False,True,True,"I'm sorry, but it seems like your query didn't..."
2,i,Sensex,2024-01-17T00:15:05,You are an Indian finance and business expert ...,|| user_query: Sensex || today date: 17 Janua...,gpt-4-0125-preview,"""**Quick Insights**\n\nAs of the latest update...",3363,92,False,**Quick Insights**<br><br>As of the latest upd...,Quick Insights,"As of the latest update on December 18, 2023, ...","As of the latest update on December 18, 2023, ...",False,True,False,False,"As of the latest update on December 18, 2023, ..."
3,i,Best shawarma in India,2024-01-03T22:10:44,You are an Indian finance and business expert ...,|| user_query: Best shawarma in India || toda...,gpt-4-0125-preview,"""I'm sorry, but I'm unable to fulfill this req...",5423,13,False,"I'm sorry, but I'm unable to fulfill this requ...",None,"I'm sorry, but I'm unable to fulfill this requ...","I'm sorry, but I'm unable to fulfill this requ...",True,False,False,True,"I'm sorry, but I'm unable to fulfill this requ..."
4,i,Best shawarma in India,2024-01-03T22:10:44,You are an Indian finance and business expert ...,|| user_query: Best shawarma in India || toda...,gpt-4-0125-preview,"""I'm sorry, but I don't have any information o...",5423,50,False,"I'm sorry, but I don't have any information on...",None,"I'm sorry, but I don't have any information on...","I'm sorry, but I don't have any information on...",True,False,False,True,"I'm sorry, but I don't have any information on..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,i,what is 10 10 d section,2024-01-07T10:44:19,You are an Indian finance and business expert ...,|| user_query: what is 10 10 d section || to...,gpt-4-0125-preview,"""**Quick Insights**\n\nSection 10 (10D) of the...",6088,126,False,**Quick Insights**<br><br>Section 10 (10D) of ...,Quick Insights,Section 10 (10D) of the Income Tax Act exempts...,Section 10 (10D) of the Income Tax Act exempts...,False,True,False,False,Section 10 (10D) of the Income Tax Act exempts...
1117,i,Help me select a tax regime in 2024,2024-01-04T04:08:36,You are an Indian finance and business expert ...,|| user_query: Help me select a tax regime in...,gpt-4-0125-preview,"""Choosing between the old and new income tax r...",2381,309,False,Choosing between the old and new income tax re...,Quick Insights,- **New Tax Regime (From FY 2023-24 onwards)**...,- **New Tax Regime (From FY 2023-24 onwards)**...,False,True,False,False,- **New Tax Regime (From FY 2023-24 onwards)**...
1118,i,Wint blogs,2024-01-10T22:59:23,You are an Indian finance and business expert ...,|| user_query: Wint blogs || today date: 10 J...,gpt-4-0125-preview,"""**Quick Insights**\n\nWint Wealth, an online ...",2468,153,False,"**Quick Insights**<br><br>Wint Wealth, an onli...",Quick Insights,"Wint Wealth, an online bond platform, is expan...","Wint Wealth, an o

In [82]:
responses_df = responses_df.drop(columns = [0, 'hash'])

In [83]:
responses_df[responses_df.flagged == False].to_csv('gpt-4-cleaned_responses_v1.csv',index=False)

In [54]:
import re

def add_space_after_unicode_representation(text):
    # Pattern to find any Unicode representation (e.g., \u20b9 or \U000020b9) followed directly by digits.
    # This pattern matches both short (\uxxxx) and long (\Uxxxxxxxx) Unicode representations.
    pattern = r'(\\u[0-9a-fA-F]{4}|\\U[0-9a-fA-F]{8})(\d)'

    # Replacement pattern with a space between the Unicode representation and the number.
    replacement = r'\1 \2'

    # Use re.sub to replace the pattern with our desired format.
    modified_text = re.sub(pattern, replacement, text)

    return modified_text

In [47]:
import json

In [75]:
(responses_df['input_tokens'].sum() * 0.01)/1000 + (responses_df['output_tokens'].sum() * 0.03)/1000

47.66874

In [76]:
responses_df['output_tokens'].describe()

count    1121.000000
mean      198.030330
std       131.018691
min         9.000000
25%        92.000000
50%       157.000000
75%       299.000000
max       641.000000
Name: output_tokens, dtype: float64